In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

In [2]:
# import matplotlib
# matplotlib.rcParams.update({'font.size':18})
# matplotlib.rcParams.update({'font.family':'serif'})

# https://github.com/dhaitz/mplcyberpunk
import mplcyberpunk
plt.style.use("cyberpunk")

## Puzzle 1

In [65]:
df = pd.read_table('input.txt', delimiter='|', names=['input', 'output'])
# df = pd.read_table('test.txt', delimiter='|', names=['input', 'output'])
df

,input,output
0,begfcd fabg aecgbdf cefagb edgcba eacbf efgbc ...,cbgef befdcg bceaf fagb
1,cfdegb bdefca ac becdf cefagd bfac cfegbad aeg...,ca dgfaec gbdae agcdfe
2,dfegbc dea cabdefg efdgca fgabed dgeac cgadb g...,dfecg feca bgcad ebgfcd
3,cdbag ca gdbae cga acfd cabdefg bgfcad eabcgf ...,cagbd gadcb gacbd dbagfc
4,facedg begfac efdga ag cefadbg gacd bdgefc afg...,fag ebcgfd gfeda afg
...,...,...
195,gf fadce fag bcgad bcfg fdcga abdfeg gdfacb de...,cdaegb feacd fbgc eafdcgb
196,ebgdca fedgab fdcab acdegfb cabdfe fda acbde a...,bfgeda deabc bdeca ecadb
197,fbedcg cf cabdg dgcebaf cfg efcb aegfcd fbdcg ...,ebcf fc gbfcd cdfbge
198,def fd fedga ebdcaf fgbd cbgeafd cabdeg edfbga...,cagdbe fbgaecd afgce fde


In [67]:
# we need to find how many entries have "unique values", so lengths of
# [2,3,4,7], corresponding to digits [1,7,4,8] respectively

dfcnt = df['output'].str.strip().str.split(expand=True)#.apply(lambda x : len(x))
# tried some pandas tricks... couldnt get str len to work across columns... just do it manually 

for k in range(dfcnt.shape[1]):
    # count the length of the strings
    dfcnt[k] = dfcnt[k].str.len()
    # see if they're in the magic list
    dfcnt[k] = dfcnt[k].apply(lambda x : x in [2,3,4,7])

dfcnt.sum(axis=0).sum()

367

## Puzzle 2

In [71]:
df = pd.DataFrame(data={'input':['acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab '], 
                        'output':[' cdfeb fcadb cdfeb cdbaf']})
# df = pd.read_table('test.txt', delimiter='|', names=['input', 'output'])
# df = pd.read_table('input.txt', delimiter='|', names=['input', 'output'])

# now lets decode the output column
# df['output'].str.strip().str.split(expand=True)
df

,input,output
0,acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb ea...,cdfeb fcadb cdfeb cdbaf


## This is one approach... now code it!

acedgfb: 8
ab: 1
dab: 7, A=d
eafb: 4

[2,3,5]
cdfbe: 5
gcdfa: 2
fbcad:  3
[0,6,9]
cefabd: 9
cdfgeb: 6
cagedb: 0

9 is 4 + A + X, cefabd
3 is 1 + A + G + X, abdcf
0 has no D, cagedb
6 the last 6-digit one, solving C/F degeneracy

can then read off 2,5

````  
 AAAA    dddd
B    C  e    a
B    C  e    a
 DDDD    ffff
E    F  g    b
E    F  g    b
 GGGG    cccc
````

In [186]:
def SOLVEIT(input):
    # input = df['input'].str.strip()[k].split(' ')
    ins = np.array(input, dtype=str)
    nums = pd.Series(input).str.len()
    # ins =   np.array(df['input'].str.strip()[0].split(' '), dtype=str)
    # nums = pd.Series(df['input'].str.strip()[0].split(' ')).str.len()
    # 1,7,4,8 are determined by str len directly

    n1 = ins[nums == 2][0]
    n7 = ins[nums == 3][0]
    n4 = ins[nums == 4][0]
    n8 = ins[nums == 7][0]

    x5 = np.where(nums == 5)[0] # [2,3,5]
    x6 = np.where(nums == 6)[0] # [0,6,9]

    # find A
    # loop over digits of 7
    for k in range(3):
        # which digit of 7 isnt in 1
        if not n7[k] in n1:
            # that digit is A
            A = n7[k]

    # find the number 9: 4 + A + (G)
    for j in x6: # look at all 6-digit entries
        # test if they contain the digits of 4 and A
        if n4[0] in ins[j] and n4[1] in ins[j] and n4[2] in ins[j] and n4[3] in ins[j]:
            n9 = ins[j]
            for k in range(len(n9)):
                if not n9[k] in n4+A:
                    G = n9[k]

    # find the number 3: 1+A+G+X
    for j in x5:
        if (n1[0] in ins[j] and n1[1] in ins[j] and A in ins[j] and G in ins[j]):
            n3 = ins[j]
            for k in range(len(n3)):
                if not n3[k] in n1+A+G:
                    D = n3[k]

    # find 0, the only entry w/o a D
    for j in x6:
        if not D in ins[j]:
            n0 = ins[j]

    # find 6, the only remaining 6-digit entry
    n6 = ins[x6[np.where((ins[x6] != n0) & (ins[x6] != n9))[0]]][0]
    # now solve the CF degeneracy from 1 w/ 6
    if n1[0] in n6:
        C = n1[1]
        F = n1[0]
    else:
        C = n1[0]
        F = n1[1]

    # now find 2 (has a C) and 5 (has a F)
    for j in x5:
        if (not n3 in ins[j]) and (C in ins[j]):
            n2 = ins[j]
        if (not n3 in ins[j]) and (F in ins[j]):
            n5 = ins[j]

    soln = [n0, n1, n2, n3, n4, n5, n6, n7, n8, n9]
    return soln


def APPLYIT(output, soln):
    # out = pd.Series(df['output'].str.strip()[0].split(' '))
    NUMS = np.arange(10)
    VALUES = ''
    # step thru each output entry
    for j in range(len(output)):
        prob = np.ones(len(soln), dtype=bool)
        entry = output[j]
        prob = prob * pd.Series(soln).str.len().values == len(entry)
        for k in range(len(entry)): # len of the entry
            prob = prob * pd.Series(soln).str.contains(entry[k]).values
        VALUES = VALUES + str(NUMS[prob][0])
    return int(VALUES)

In [259]:
soln = SOLVEIT(df['input'].str.strip()[0].split(' '))
APPLYIT(df['output'].str.strip()[0].split(' '), soln)

5353

In [262]:
df = pd.read_table('input.txt', delimiter='|', names=['input', 'output'])
bigout = np.zeros(len(df))
for i in range(len(df)):
    soln = SOLVEIT(df['input'].str.strip()[i].split(' '))
    bigout[i] = APPLYIT(df['output'].str.strip()[i].split(' '), soln)

In [263]:
bigout

array([5634., 1020., 5426., 3339., 7637., 9451., 5194., 5112., 8427.,
       5668., 9378., 9876., 8387., 1975., 6645., 2814., 7846., 8450.,
       3348., 6984., 9174., 8016., 6031., 2585., 7638., 3181., 8645.,
        370., 9789., 4414., 1771., 1551., 4867., 8078., 4966., 4256.,
       1817., 9383., 1342., 4731., 7525.,  696., 1486., 2727., 2987.,
       3724., 5417., 7553., 8989., 1489., 6341., 7289., 4007., 6901.,
       6444., 7317., 6586., 1026., 4612., 2574., 7221.,  409., 3238.,
       3519., 8654.,  687.,  776.,  170., 4462., 5233., 7880., 1256.,
       7306., 1041., 4024., 2332., 9820., 4837.,  246., 9410., 4584.,
        588., 2453., 7934., 3215., 6594., 6519., 6158., 9180., 9039.,
       9711., 3785., 5054., 3055., 6051., 1473., 2064., 9938., 7506.,
       7251., 4317., 3467., 9773., 1239., 1317., 8027., 1650., 6849.,
       5706., 9965., 1921., 2789., 6154., 1912., 4967., 1844., 3255.,
       5478., 2949., 3623., 2645., 5748., 4884., 9747., 4932., 6774.,
       6103., 1507.,

In [264]:
np.sum(bigout)

974512.0